In [3]:
import pandas as pd
from datetime import datetime

# Load the datasets
transactions_df = pd.read_csv('./Raw_Data/TRANSACTION_TAKEHOME.csv')
users_df = pd.read_csv('./Raw_Data/USER_TAKEHOME.csv')
products_df = pd.read_csv('./Raw_Data/PRODUCTS_TAKEHOME.csv')

## Close ended Questions

 1. What are the top 5 brands by receipts scanned among users 21 and over?

 - SQL Query:

```sql
-- Find the top 5 brands by receipts scanned among users 21 and over
SELECT
    p.brand,                      
    COUNT(t.receipt_id) AS receipts_scanned   
FROM
    Transactions t                
JOIN
    Users u                      
ON 
    t.user_id = u.id              -- Joining condition based on user_id
JOIN 
    Products p                    
ON 
    t.barcode = p.barcode          -- Joining condition based on barcode
WHERE
    DATEDIFF(CURDATE(), u.birth_date) / 365 >= 21  -- Filter users 21 or older
GROUP BY
    p.brand                        -- Grouping by product brand
ORDER BY 
    receipts_scanned DESC          -- Sorting by number of receipts scanned in descending order
LIMIT 5;                           -- Limiting the result to top 5 brands



In [7]:
# Convert BIRTH_DATE to datetime, and ensure it's timezone-naive
users_df['BIRTH_DATE'] = pd.to_datetime(users_df['BIRTH_DATE'], errors='coerce').dt.tz_localize(None)

# Get the current date as timezone-naive
current_date = datetime.now()

# Calculate the age of the users and filter for those 21 or older
users_df['AGE'] = (current_date - users_df['BIRTH_DATE']).dt.days // 365
users_21_plus = users_df[users_df['AGE'] >= 21]

# Merge transactions with users who are 21 or older
merged_df = pd.merge(transactions_df, users_21_plus, left_on='USER_ID', right_on='ID', how='inner')

# Merge the result with the products dataframe on the barcode
final_df = pd.merge(merged_df, products_df, left_on='BARCODE', right_on='BARCODE', how='inner')

# Group by brand and count the number of receipts scanned
top_brands = final_df.groupby('BRAND')['RECEIPT_ID'].count().sort_values(ascending=False).head(5)

# Display the top 5 brands by number of receipts scanned
print(top_brands)


BRAND
COCA-COLA                    628
ANNIE'S HOMEGROWN GROCERY    576
DOVE                         558
BAREFOOT                     552
ORIBE                        504
Name: RECEIPT_ID, dtype: int64


2. What is the percentage of sales in the Health & Wellness category by generation?

- SQL Query:

``` sql
-- Define a Common Table Expression (CTE) to categorize users into generations based on their birth date
WITH UserAge AS (
    SELECT 
        u.id, 
        CASE 
            WHEN DATEDIFF(CURDATE(), u.birth_date) / 365 < 27 THEN 'Gen Z'
            WHEN DATEDIFF(CURDATE(), u.birth_date) / 365 BETWEEN 27 AND 42 THEN 'Millennials'
            WHEN DATEDIFF(CURDATE(), u.birth_date) / 365 BETWEEN 43 AND 58 THEN 'Gen X'
            ELSE 'Boomers'
        END AS generation
    
    FROM 
        Users u
)
-- Query to calculate the percentage of sales in the 'Health & Wellness' category for each generation
SELECT 
    ua.generation, 
    SUM(CASE WHEN p.category_1 = 'Health & Wellness' THEN t.sale ELSE 0 END) / SUM(t.sale) * 100 AS health_sales_percentage -- Calculate the percentage of sales in the 'Health & Wellness' category

FROM 
    Transactions t
JOIN 
    UserAge ua ON t.user_id = ua.id -- Join the Transactions table with the UserAge CTE to get user generation
JOIN 
    Products p ON t.barcode = p.barcode -- Join with Products to get product categories

GROUP BY 
    ua.generation; -- Group the results by generation to get the percentage for each group



In [13]:
# Step 1: Convert BIRTH_DATE to datetime format and ensure it's timezone-naive
users_df['BIRTH_DATE'] = pd.to_datetime(users_df['BIRTH_DATE'], errors='coerce').dt.tz_localize(None)

# Step 2: Convert FINAL_SALE to numeric, handling errors
transactions_df['FINAL_SALE'] = pd.to_numeric(transactions_df['FINAL_SALE'], errors='coerce')

# Step 3: Get the current date
current_date = datetime.now()

# Step 4: Calculate the age of the users and categorize them into generations
def categorize_generation(birth_date):
    if pd.isnull(birth_date):
        return None
    age = (current_date - birth_date).days // 365
    if age < 27:
        return 'Gen Z'
    elif 27 <= age <= 42:
        return 'Millennials'
    elif 43 <= age <= 58:
        return 'Gen X'
    else:
        return 'Boomers'

# Apply the generation categorization
users_df['GENERATION'] = users_df['BIRTH_DATE'].apply(categorize_generation)

# Step 5: Merge users with transactions to link users and their generations with transactions
merged_df = pd.merge(transactions_df, users_df[['ID', 'GENERATION']], left_on='USER_ID', right_on='ID', how='inner')

# Step 6: Merge the result with the products dataframe to get product categories
final_df = pd.merge(merged_df, products_df[['BARCODE', 'CATEGORY_1']], left_on='BARCODE', right_on='BARCODE', how='inner')

# Step 7: Calculate the percentage of sales in 'Health & Wellness' category for each generation

# Filter for the 'Health & Wellness' category and calculate sales for this category
health_wellness_sales = final_df[final_df['CATEGORY_1'] == 'Health & Wellness'].groupby('GENERATION')['FINAL_SALE'].sum()

# Calculate total sales for each generation
total_sales_by_generation = final_df.groupby('GENERATION')['FINAL_SALE'].sum()

# Step 8: Calculate the percentage of health and wellness sales for each generation
health_sales_percentage = (health_wellness_sales / total_sales_by_generation) * 100

# Display the result
print(health_sales_percentage)


GENERATION
Boomers        57.252918
Gen X          57.248801
Millennials    57.278642
Name: FINAL_SALE, dtype: float64


In [10]:
print(transactions_df.columns)

Index(['RECEIPT_ID', 'PURCHASE_DATE', 'SCAN_DATE', 'STORE_NAME', 'USER_ID',
       'BARCODE', 'FINAL_QUANTITY', 'FINAL_SALE'],
      dtype='object')


## Open Ended Questions

1. Who are Fetch’s power users?

- SQL Query:

``` sql

-- Find Fetch's power users based on receipts scanned and total sales
-- A "power user" is defined as a user who has scanned more than 100 receipts

SELECT 
    u.id, 
    COUNT(t.receipt_id) AS total_receipts,  -- Count the number of receipts (transactions) per user
    SUM(t.sale) AS total_sales  -- Sum up the total sales for each user
FROM 
    Transactions t  
JOIN 
    Users u 
ON 
    t.user_id = u.id  -- Join the Transactions and Users tables on the user ID
GROUP BY 
    u.id  -- Group the results by user ID so we can aggregate data (receipts and sales) for each user
HAVING 
    COUNT(t.receipt_id) > 100  -- Filter to include only users who have more than 100 receipts
ORDER BY 
    total_sales DESC;  -- Sort the results in descending order of total sales, so the highest spending users appear first

In [19]:
# Step 1: Convert 'FINAL_SALE' column to numeric
# This ensures that any non-numeric values (such as text) are converted to NaN
transactions_df['FINAL_SALE'] = pd.to_numeric(transactions_df['FINAL_SALE'], errors='coerce')

# Step 2: Merge transactions with users
merged_df = pd.merge(transactions_df, users_df, left_on='USER_ID', right_on='ID', how='inner')

# Step 3: Group by 'user_id' to calculate total receipts and total sales for each user
# - 'total_receipts' is the count of receipts (i.e., how many transactions the user has)
# - 'total_sales' is the sum of all sales made by that user
user_stats = merged_df.groupby('USER_ID').agg(
    total_receipts=('RECEIPT_ID', 'count'),  # Count the number of receipts for each user
    total_sales=('FINAL_SALE', 'sum')  # Sum up the sales for each user
).reset_index()  # Reset the index to turn 'user_id' into a regular column

# Step 4: Filter for power users with more than 100 receipts
power_users = user_stats[user_stats['total_receipts'] > 100]

# Step 5: Sort the power users by total sales in descending order
# This helps identify the users who have spent the most in total
power_users_sorted = power_users.sort_values(by='total_sales', ascending=False)

# Display the top power users sorted by total sales
print(power_users_sorted)

Empty DataFrame
Columns: [USER_ID, total_receipts, total_sales]
Index: []
